In [1]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import re
import time 
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def get_html(url):
    response = requests.get(url)
    return soup(response.text)

In [3]:
fechas = pd.date_range(start="2024-01-01", end="2024-12-31", freq="D")
final_df = pd.DataFrame()
current_year = fechas[0].year  # Inicializa el primer año
start_time = time.time()

for fecha in fechas:
    try:
        url = "http://rmcab.ambientebogota.gov.co/Report/HourlyReports?id=1&UserDateString=" + str(fecha.date())
        html = get_html(url)
        time.sleep(1)
        
        tablas = html.find_all("div", class_="TableStation")
        for tabla in tablas:
            estacion = tabla.find("h2").text.strip()
            table = tabla.find("table")
            rows = tabla.find_all("tr")

            table_data = []

            for row in rows:
                columns = row.find_all(["td", "th"])
                columns = [col.get_text(strip=True) for col in columns]
                table_data.append(columns)

            df = pd.DataFrame(table_data)

            df.columns = df.iloc[0]
            df = df[1:]  
            df["Fecha"] = fecha
            df["Estación"] = estacion
            final_df = pd.concat([final_df, df], ignore_index=True)

        if fecha.year != current_year:

            end_time = time.time()
            elapsed_time = end_time - start_time

            #final_df.to_excel(f"../Datos/RMCAB/Raw/Raw_RMCAB_{current_year}.xlsx", index=False)
            final_df.to_excel(f"./Raw_RMCAB_{current_year}.xlsx", index=False)
            print(f"Guardado archivo para el año {current_year}")
            print(f"Tiempo transcurrido para descargar los datos de {current_year}: {elapsed_time/60:.2f} minutos")

            final_df = pd.DataFrame()
            current_year = fecha.year
            start_time = time.time() 
    except Exception as e:
        print(f"Error en {fecha}: {e}")
        continue

if not final_df.empty:
    end_time = time.time()
    elapsed_time = end_time - start_time
    final_df.to_excel(f"./Raw_RMCAB_{current_year}.xlsx", index=False)

In [7]:
#final_df.to_csv(f"../Datos/Raw_RMCAB_{2025}.csv", index=False)
#final_df.to_excel(f"../Datos/Raw_RMCAB_{2025}.xlsx", index=False)